In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.5 MB/s eta 0:00:00


In [2]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

# Ensure required package is installed
try:
    import datasets
except ImportError:
    import subprocess
    subprocess.run(["pip", "install", "datasets"], check=True)
    from datasets import Dataset

# Function to compute accuracy, precision, recall, and F1-score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = np.mean(predictions == labels)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    loss = torch.nn.functional.cross_entropy(torch.tensor(logits), torch.tensor(labels)).item()
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, "loss": loss}

# Load dataset
df = pd.read_csv("/content/final_labels.csv")

# Check column names
print("Columns in dataset:", df.columns)

# Ensure correct column names
df.rename(columns=lambda x: x.strip().lower(), inplace=True)

# Select text and label columns
if 'body' not in df.columns or 'group' not in df.columns:
    raise KeyError("Required columns 'body' and 'group' not found in dataset. Available columns: " + str(df.columns))

df = df[['body', 'group']].rename(columns={'body': 'text', 'group': 'label'})

# Drop any rows with NaN values
df = df.dropna()

# Convert text to string and labels to integer
df['text'] = df['text'].astype(str)

# Map text labels to integers if necessary
label_mapping = {label: idx for idx, label in enumerate(df['label'].unique())}
df['label'] = df['label'].map(label_mapping).astype(int)

# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)

# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Convert to Hugging Face Dataset format
dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
dataset = dataset.map(tokenize_function, batched=True)
dataset = dataset.train_test_split(test_size=0.2)

# Load model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(label_mapping))

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_metrics
)

# Train model
trainer.train()

# Evaluate model
metrics = trainer.evaluate()
print(f"Final Evaluation Results - Accuracy: {metrics['eval_accuracy']}, Precision: {metrics['eval_precision']}, Recall: {metrics['eval_recall']}, F1 Score: {metrics['eval_f1']}, Loss: {metrics['eval_loss']}")

# Save model
model.save_pretrained("./subreddit_classifier")
tokenizer.save_pretrained("./subreddit_classifier")

print("Model training complete. Saved to ./subreddit_classifier")

Columns in dataset: Index(['entry_id', 'link_id', 'parent_id', 'entry_utc', 'subreddit', 'author',
       'body', 'image', 'label_date', 'week', 'group', 'sheet_order',
       'level_1', 'level_2', 'level_3', 'strength', 'highlight', 'split'],
      dtype='object')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 422264 (422264-national-institute-of-technology-andhrapradesh) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.694300,0.697971,0.458532,0.210252,0.458532,0.288306
2,0.701800,0.690980,0.541468,0.293188,0.541468,0.380401
3,0.692500,0.690074,0.536702,0.454081,0.536702,0.388046
4,0.694000,0.715971,0.510010,0.533779,0.510010,0.493668
5,0.615700,0.777402,0.599619,0.597171,0.599619,0.596797
6,0.481100,0.987016,0.606292,0.604623,0.606292,0.604856
7,0.360000,1.369270,0.571973,0.578545,0.571973,0.572323
8,0.254800,1.706174,0.586273,0.595020,0.586273,0.586100
9,0.200700,1.812131,0.596759,0.594682,0.596759,0.594807
10,0.164200,1.939029,0.596759,0.599247,0.596759,0.597414


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Final Evaluation Results - Accuracy: 0.5967588179218303, Precision: 0.5992469922903167, Recall: 0.5967588179218303, F1 Score: 0.5974135656281436, Loss: 1.9390286207199097
Model training complete. Saved to ./subreddit_classifier
